In [1]:
include("load_pod.jl");
include("load_s1slc_ann.jl");

In [2]:
f = open("ann_path.txt")
path = readlines(f)
sar_parameters = load_s1slc_ann(path[1])

f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],sar_parameters["t_0"]);

In [3]:
llh = [
    [(56 + 33/60 + 42.5556/(60*60)),(8 + 18/60 + 26.874/(60*60)), 42.753],
    [(56 + 33/60 + 44.4528/(60*60)) ,(8 + 18/60 + 24.865/(60*60)),43.282],
    [(56 + 33/60 + 42.8652/(60*60)) ,(8 + 18/60 + 22.104/(60*60)),44.475],
    [(56.003092) ,8.129702 ,43.5]
]

4-element Array{Array{Float64,1},1}:
 [56.5618, 8.30747, 42.753]
 [56.5623, 8.30691, 43.282]
 [56.5619, 8.30614, 44.475]
 [56.0031, 8.1297, 43.5]   

In [6]:
include("llh2sali.jl")

llh2sali

In [7]:
llh2sali(llh, osv, t_sv, sar_parameters)

4-element Array{Array{Float64,1},1}:
 [1126.43, 6086.27]
 [1122.66, 6098.7] 
 [1126.71, 6109.43]
 [5685.96, 6255.93]

In [36]:
include("zero_doppler_bisect.jl")
include("ellipsoid2xyz.jl");
include("calc_sat_trajectory.jl");
include("load_pod.jl");

In [41]:
c = 299792458 # speed of light
    
t_0 = sar_parameters["t_0"]
t_start = sar_parameters["t_start"]
t_stop = sar_parameters["t_stop"]

range_pixel_spacing =  sar_parameters["range_pixel_spacing"]
azimuth_frequency =  sar_parameters["azimuth_frequency"] 
r_near =  sar_parameters["slant_range_time"]  *c/2

osv_poly, osv_mean, osv_std = calc_sat_trajectory(osv, t_sv, t_start, t_stop);

In [48]:
sali = [Array{Float64}(undef,2) for point in llh]
point_xyz = [ellipsoid2xyz(point[1]* pi/180, point[2]* pi/180, point[3]) for point in llh]

for i in 1:length(llh)
    range, time = zero_doppler_bisect(point_xyz[i], t_start, t_stop,
                                    osv_poly, osv_mean, osv_std)
    sali[i][1] = 1 + (time - t_start) * azimuth_frequency
    sali[i][2] = 1 + (range - r_near) / range_pixel_spacing
end



In [49]:
sali

4-element Array{Array{Float64,1},1}:
 [1126.43, 6086.27]
 [1122.66, 6098.7] 
 [1126.71, 6109.43]
 [5685.96, 6255.93]

In [39]:
point_xyz[1][1]

3.4858680388680836e6